In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib

from scipy.linalg import expm, null_space

plt.style.use("ggplot")
plt.rcParams["font.family"] = "sans-serif"
sns.set_palette("dark")
sns.set_context("talk")

import numpy as np

from IPython.display import HTML, display, SVG

import pydot
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, BasicVector,
                        Parser, Saturation, Simulator, PlanarSceneGraphVisualizer, 
                        LinearQuadraticRegulator, AbstractValue, MeshcatVisualizer, 
                        StartMeshcat, WrapToSystem, SceneGraph, DiscreteTimeLinearQuadraticRegulator,
                        JacobianWrtVariable, eq, SnoptSolver)    

from pydrake.systems.primitives import LogVectorOutput, ConstantVectorSource, ZeroOrderHold
from pydrake.multibody.inverse_kinematics import PositionConstraint, AddUnitQuaternionConstraintOnPlant

from pydrake.multibody.plant import ContactResults, CoulombFriction
from pydrake.autodiffutils import AutoDiffXd, InitializeAutoDiff, ExtractGradient, ExtractValue
from pydrake.math import RollPitchYaw_, RotationMatrix_, RigidTransform_
from underactuated.meshcat_cpp_utils import MeshcatSliders, MeshcatJointSliders
from Custom_LeafSystems import *

from tqdm import tqdm as tqdm

from Linearize import getGradients
from pydrake.multibody import plant as plnt
plnt.ContactModel(1)

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve

from datetime import datetime

from lin_control_tools import controllability_matrix, observability_matrix, controllability, observability, stabilizability, detectability


In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

In [ ]:
time_step = 0.0
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
body = Parser(plant).AddModelFromFile("Double_Leg/Double_Leg_Collisions.urdf")
plant.Finalize()

meshcat.Delete()
vis = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
context = plant.CreateDefaultContext()

num_pos = plant.num_positions()
num_vel = plant.num_velocities()
num_act = plant.num_actuators()
# num_disc_states = context.num_total_states() - context.num_continuous_states()
num_disc_states = context.num_continuous_states()
fb_num_pos = 7
fb_num_vel = 6
n = 6

x0 = np.zeros(num_disc_states)
x0[0], x0[6] = 1, 0.4

plant.SetPositionsAndVelocities(context, x0)

CoM_0 = plant.CalcCenterOfMassPositionInWorld(context)

plant.get_actuation_input_port().FixValue(context, np.zeros(num_act))

# display(SVG(pydot.graph_from_dot_data(plant.GetTopologyGraphvizString())[0].create_svg()))

foot_L = ["collision_dummy_L_BR", "collision_dummy_L_BL", "collision_dummy_L_FR", "collision_dummy_L_FL"]
foot_R = ["collision_dummy_R_BR", "collision_dummy_R_BL", "collision_dummy_R_FR", "collision_dummy_R_FL"]
feet = ["foot_L", "foot_R"]
feet_center = ["collision_dummy_L_C", "collision_dummy_R_C"]
foot_vertices = ["collision_dummy_L_BR", "collision_dummy_L_BL", "collision_dummy_L_FR", "collision_dummy_L_FL", 
                "collision_dummy_R_BR", "collision_dummy_R_BL", "collision_dummy_R_FR", "collision_dummy_R_FL"]



num_contact = len(foot_L) + len(foot_R)

def sethome(plant, context, ad=1.):

    shaft_roll = 0.3
    hip_pitch = 0.3
    knee_pitch = 0.6
    foot_pitch = 0.3

    plant.GetJointByName("shaft_roll").set_angle(context, shaft_roll)
    plant.GetJointByName("Hip_Pitch_R").set_angle(context, hip_pitch)
    plant.GetJointByName("Hip_Pitch_L").set_angle(context, hip_pitch)
    plant.GetJointByName("knee_R").set_angle(context, -knee_pitch)
    plant.GetJointByName("knee_L").set_angle(context, -knee_pitch)
    plant.GetJointByName("foot_leg_pitch_R").set_angle(context, foot_pitch)
    plant.GetJointByName("foot_leg_pitch_L").set_angle(context, -foot_pitch)

    foot_pos = plant.GetBodyByName("collision_dummy_L_C").EvalPoseInWorld(context).GetAsMatrix34()[:, -1].reshape(3)
    body_pos = plant.GetBodyByName("base_link").EvalPoseInWorld(context).GetAsMatrix34()[:, -1].reshape(3)

    body_pos[2] = body_pos[2] - foot_pos[2] + 0.001 # Add radius of collision sphere
    body_pos[1] = -foot_pos[1]

    if np.array([ad]).dtype == float:
        transform = RigidTransform_[float](body_pos)
    else:
        transform = RigidTransform_[AutoDiffXd](body_pos)

    plant.SetFreeBodyPose(context, plant.GetBodyByName("base_link"), transform)


# sethome(plant, context)

x0 = plant.GetPositionsAndVelocities(context)

# print(plant.GetBodyByName(foot_L).EvalPoseInWorld(context))
# display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=2))[0].create_svg()))

diagram = builder.Build()

def PoseOptimization():
    context_diagram = diagram.CreateDefaultContext()
    plant_context_from_diagram = diagram.GetSubsystemContext(plant, context_diagram)
    plant_context_from_diagram.get_mutable_discrete_state_vector().SetFromVector(x0)

    sliders = MeshcatJointSliders(meshcat, plant, context_diagram)
    sliders.Run(vis, context_diagram)
    meshcat.DeleteAddedControls()
    meshcat.Delete()

PoseOptimization()


In [ ]:

# Pose optimization

plant_f = plant
context_f = plant_f.CreateDefaultContext()

plant_ad = plant.ToAutoDiffXd()
context_ad = plant_ad.CreateDefaultContext()

plant_f.SetPositionsAndVelocities(context_f, x0)
plant_ad.SetPositions(context_ad, InitializeAutoDiff(x0[:num_pos]))
plant_ad.get_actuation_input_port().FixValue(context_ad, InitializeAutoDiff(np.zeros(num_act)))

# plant_ad.SetPositions(context_ad, InitializeAutoDiff(x0))

B = plant.MakeActuationMatrix()

def makePosConstraint(plant, context):
    # Position constraint feet
    
    p_AQ_lower = np.array([-np.inf, -np.inf, 0.001])
    p_AQ_upper = np.array([np.inf, np.inf, 0.001])
    p_BQ = np.zeros(3)

    frame_world = plant.world_frame()
    constraints = []

    for point in foot_L:
        frame_foot = plant.GetFrameByName(point)

        pos_constraint = PositionConstraint(plant=plant,
                            frameA=frame_world,
                            p_AQ_lower=p_AQ_lower,
                            p_AQ_upper=p_AQ_upper,
                            frameB=frame_foot,
                            p_BQ=p_BQ,
                            plant_context=context
                            )

        constraints.append(pos_constraint)

    for point in foot_R:
        frame_foot = plant.GetFrameByName(point)

        pos_constraint = PositionConstraint(plant=plant,
                            frameA=frame_world,
                            p_AQ_lower=p_AQ_lower,
                            p_AQ_upper=p_AQ_upper,
                            frameB=frame_foot,
                            p_BQ=p_BQ,
                            plant_context=context
                            )

        constraints.append(pos_constraint)

    return constraints

pos_constraint = makePosConstraint(plant_ad, context_ad)

In [ ]:
def get_foot_Jacobian(var):

    if var.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad 

    plant.SetPositions(context, var)
    frame_world = plant.world_frame()

    p_BoBi_B = np.zeros(3)

    for index, point in enumerate(feet_center):
        
        frame_foot = plant.GetFrameByName(point)

        Jc_sp = plant.CalcJacobianSpatialVelocity(
                context,
                JacobianWrtVariable(1),
                frame_foot,
                p_BoBi_B,
                frame_world,
                frame_world
                )

        if index == 0:
            Jc = Jc_sp
        else:
            Jc = np.vstack((Jc, Jc_sp))

    return Jc


def calcUnactuatedDynamics(vars):

    split_at = [num_act, num_act + num_pos]
    joint_torque, pos, lambda_c = np.split(vars, split_at)
    
    if joint_torque.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad 

    plant.SetPositions(context, pos)
    plant.SetVelocities(context, np.zeros(num_vel))
    plant.get_actuation_input_port().FixValue(context, joint_torque)

    # M, Cv, gravity_forces
    gravity_forces = plant.CalcGravityGeneralizedForces(context)
    Jc = get_foot_Jacobian(pos)

    return gravity_forces + B @ joint_torque + Jc.T @ lambda_c


def CostTorque(joint_torque):
    return joint_torque.dot(joint_torque)


def CostCoM(q):

    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad

    plant.SetPositions(context, q)
    CoM = plant.CalcCenterOfMassPositionInWorld(context)

    CoM_xy = CoM[0:2]
    CoM_z = CoM[2]

    return CoM_xy.dot(CoM_xy) + 1 / (w_CoM*CoM_z)

def GetCoM(q):

    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad

    # plant.SetPositions(context, q)
    CoM = plant.CalcCenterOfMassPositionInWorld(context)[0:2]

    return CoM


def calcStaticEq(f_c):

    if f_c.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad

    f_g = plant.gravity_field().gravity_vector() * plant.CalcTotalMass(context)

    # if isinstance(f_g, AutoDiffXd):
    #     f_g = ExtractValue(f_g).flatten()

    return f_c[3:6] + f_c[9:] + f_g


def calcMomentBal(f_c):

    if f_c.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad

    CoM = plant.CalcCenterOfMassPositionInWorld(context)
    C_left = plant.GetBodyByName("collision_dummy_L_C").EvalPoseInWorld(context).GetAsMatrix34()[:, -1].reshape(3)
    C_right = plant.GetBodyByName("collision_dummy_R_C").EvalPoseInWorld(context).GetAsMatrix34()[:, -1].reshape(3)

    # CoM_L = ExtractValue(C_left - CoM).flatten()
    # CoM_R = ExtractValue(C_right - CoM).flatten()

    CoM_L = C_left - CoM
    CoM_R = C_right - CoM

    tau_G_L = np.cross(CoM_L, f_c[3:6])
    tau_G_R = np.cross(CoM_R, f_c[9:])

    return f_c[0:3] + f_c[6:9] + tau_G_L + tau_G_R


def fc_Cost(f_c):

    return f_c[:3].dot(f_c[:3]) + f_c[6:9].dot(f_c[6:9])

    

In [ ]:
# Write Mathematical program
prog = MathematicalProgram()

# Define decision variables
pos = prog.NewContinuousVariables(num_pos, "positions")
torques = prog.NewContinuousVariables(num_act, "torques")
contact_f = prog.NewContinuousVariables(2*n, "contact_forces")

# Bent knee and hip
knee_constraint = prog.AddConstraint(pos[4 + fb_num_pos] <= -0.3).evaluator().set_description("Knee bend constraint")
hip_constraint = prog.AddConstraint(pos[3 + fb_num_pos] >= 0.15).evaluator().set_description("Hip bend constraint")
knee_constraint_L = prog.AddConstraint(pos[9 + fb_num_pos] <= -0.3).evaluator().set_description("Knee bend constraint")
hip_constraint_L = prog.AddConstraint(pos[8 + fb_num_pos] >= 0.15).evaluator().set_description("Hip bend constraint")

# Unit quaternion constraint
unit_quat = AddUnitQuaternionConstraintOnPlant(plant_ad, pos, prog)

# Limit Inividual parts of quaternion to be between -1 and 1
(prog.AddBoundingBoxConstraint(-1.0, 1.0, pos[0]).evaluator().set_description("quat0 limit constraint"))
(prog.AddBoundingBoxConstraint(-1.0, 1.0, pos[1]).evaluator().set_description("quat1 limit constraint"))
(prog.AddBoundingBoxConstraint(-1.0, 1.0, pos[2]).evaluator().set_description("quat2 limit constraint"))
(prog.AddBoundingBoxConstraint(-1.0, 1.0, pos[3]).evaluator().set_description("quat3 limit constraint"))

# Static coefficient of friction
mu = 1
X = 0.06
Y = 0.04

# Origin points of the foot
O_L = plant.GetBodyByName("collision_dummy_L_C").EvalPoseInWorld(context).translation()
O_R = plant.GetBodyByName("collision_dummy_R_C").EvalPoseInWorld(context).translation()

def CWC_constraint(force, torque, foot):

    # https://scaron.info/publications/icra-2015.html
    # Linear friction cone
    prog.AddConstraint(-force[0] - mu*force[2] <= 0).evaluator().set_description(f"friction cone x pos {foot}")
    prog.AddConstraint(force[0] - mu*force[2] <= 0).evaluator().set_description(f"friction cone x neg {foot}")
    prog.AddConstraint(-force[1] - mu*force[2] <= 0).evaluator().set_description(f"friction cone y pos {foot}")
    prog.AddConstraint(force[1] - mu*force[2] <= 0).evaluator().set_description(f"friction cone y neg {foot}")

    # Center of pressure constraint
    prog.AddConstraint(-Y*force[0] - torque[0] <= 0).evaluator().set_description(f"CoP x pos {foot}")
    prog.AddConstraint(-Y*force[0] + torque[0] <= 0).evaluator().set_description(f"CoP x neg {foot}")
    prog.AddConstraint(-X*force[0] - torque[1] <= 0).evaluator().set_description(f"CoP y pos {foot}")
    prog.AddConstraint(-X*force[0] - torque[1] <= 0).evaluator().set_description(f"CoP y neg {foot}")

    # Now yaw slippage
    prog.AddConstraint(-Y*force[0] - X*force[1] -(X+Y)*mu*force[2] + mu*torque[0] + mu*torque[1] - torque[2] <= 0).evaluator().set_description(f"yaw 1 {foot}")
    prog.AddConstraint(-Y*force[0] + X*force[1] -(X+Y)*mu*force[2] + mu*torque[0] - mu*torque[1] - torque[2] <= 0).evaluator().set_description(f"yaw 2 {foot}")
    prog.AddConstraint(Y*force[0] - X*force[1] -(X+Y)*mu*force[2] - mu*torque[0] + mu*torque[1] - torque[2] <= 0).evaluator().set_description(f"yaw 3 {foot}")
    prog.AddConstraint(Y*force[0] + X*force[1] -(X+Y)*mu*force[2] - mu*torque[0] - mu*torque[1] - torque[2] <= 0).evaluator().set_description(f"yaw 4 {foot}")
    prog.AddConstraint(Y*force[0] + X*force[1] -(X+Y)*mu*force[2] + mu*torque[0] + mu*torque[1] + torque[2] <= 0).evaluator().set_description(f"yaw 5 {foot}")
    prog.AddConstraint(Y*force[0] - X*force[1] -(X+Y)*mu*force[2] + mu*torque[0] - mu*torque[1] + torque[2] <= 0).evaluator().set_description(f"yaw 6 {foot}")
    prog.AddConstraint(-Y*force[0] + X*force[1] -(X+Y)*mu*force[2] - mu*torque[0] + mu*torque[1] + torque[2] <= 0).evaluator().set_description(f"yaw 7 {foot}")
    prog.AddConstraint(-Y*force[0] - X*force[1] -(X+Y)*mu*force[2] - mu*torque[0] - mu*torque[1] + torque[2] <= 0).evaluator().set_description(f"yaw 8 {foot}")


# Drake CalcJacobianSpatialVelocity returns [J_rot, J_trans] so f and tau are inverted
CWC_constraint(contact_f[3:6], contact_f[0:3], "foot_L")
CWC_constraint(contact_f[9:], contact_f[6:9], "foot_R")

# Simple CoM inside support polygon
prog.AddConstraint(GetCoM, lb=[-0.13, -0.06], ub=[0.13, 0.06], vars=contact_f)
# prog.AddConstraint(calcStaticEq, lb=-1e-5*np.ones(3), ub=1e-5*np.ones(3), vars=contact_f).evaluator().set_description("force balance")
# prog.AddConstraint(calcMomentBal, lb=-1e-5*np.ones(3), ub=1e-5*np.ones(3), vars=contact_f).evaluator().set_description("Moment balance")

# Dynamics constraints for all DOFs
vars = np.concatenate((torques, pos, contact_f))
dyn_constraint = prog.AddConstraint(calcUnactuatedDynamics, lb=[0]*(num_act + 6), ub=[0]*(num_act + 6), vars=vars)
dyn_constraint.evaluator().set_description("Dynamics constraint")

# Foot position constraints
for index, constraints in enumerate(pos_constraint):
    prog.AddConstraint(constraints, pos).evaluator().set_description(f"foot position {index}")

# Cost functions
w_CoM = 0.1
prog.AddCost(fc_Cost, vars=contact_f)

# Initial guess for contact wrench cone
fc_guess = np.zeros(n)
fc_guess[3:] = -plant.gravity_field().gravity_vector() * plant.CalcTotalMass(context) / 2
fc_guess = np.tile(fc_guess, 2)

tau_guess = np.zeros(num_act)

# Setting initial guess for decision variables
prog.SetInitialGuess(pos, x0[:num_pos])
prog.SetInitialGuess(torques, tau_guess)
prog.SetInitialGuess(contact_f, fc_guess)

# Write solver status to file
path = f"Logs/Double_Leg/{datetime.now().strftime('%Y-%m-%d_%H-%M_SNOPT.out')}"
snopt = SnoptSolver().solver_id()
prog.SetSolverOption(snopt, 'Print file', path)

# Solve optimization problem
result = Solve(prog)

# results of the optimization
q = result.GetSolution(pos)
tau = result.GetSolution(torques)
fc = result.GetSolution(contact_f)

# Second pass with cost functions
prog.SetInitialGuess(pos, q)
prog.SetInitialGuess(torques, tau)
prog.SetInitialGuess(contact_f, fc)

# cost on the joint torques and CoM position
prog.AddCost(CostTorque, vars=torques)
prog.AddCost(CostCoM, vars=pos)

result = Solve(prog)

q = result.GetSolution(pos)
tau = result.GetSolution(torques)
fc = result.GetSolution(contact_f)

x0 = np.hstack((q, np.zeros(num_vel)))
# x0[:4] = np.array([1, 0, 0, 0])

print(f'Solution found? {result.is_success()}.')

if not result.is_success():
    infeasible = result.GetInfeasibleConstraintNames(prog)
    print(infeasible)

plant.SetPositionsAndVelocities(context, x0)

CoM = plant.CalcCenterOfMassPositionInWorld(context)

# Save the optimization results to the bottom of solver output file
with open(path, "a") as file:
    file.write("\n")
    np.savetxt(file, q, delimiter=",", header="Array of robot joint angles")
    file.write("\n")
    np.savetxt(file, tau, delimiter=",", header="Array of torques")
    file.write("\n")
    np.savetxt(file, fc, delimiter=",", header="Array of contact forces")
    file.write("\n")
    np.savetxt(file, CoM, delimiter=",", header="CoM location")


In [ ]:
# Linearize system for LQR

np.set_printoptions(threshold=np.inf)

def Linearize(x0, u0, path):
    eq_state = x0
    eq_input = u0
    sys_autodiff = getGradients(eq_state, eq_input, path)
    sys_autodiff.makePlantFromURDF()
    sys_autodiff.getGradients()

    Q = np.eye(num_pos + num_vel - 1)
    
    R = 0.1*np.eye(num_act)

    # Higher weight for the x,y,z position of the floating base
    # Q[3, 3] = Q[4, 4] = Q[5, 5] = 10

    sys_autodiff.A = sys_autodiff.A[1:, 1:]
    sys_autodiff.B = sys_autodiff.B[1:, :]

    # sys_autodiff.A[:num_vel, :][sys_autodiff.A[:num_vel, :] < 0.4] = 0
    # sys_autodiff.A[(sys_autodiff.A < 1e-10) & (sys_autodiff.A > -1e-10)] = 0

    # TODO: A matrix is not full rank, find controllable realization like in LQR humanoid paper by mason et al.

    A_cont = sys_autodiff.A
    B_cont = sys_autodiff.B

    return A_cont, B_cont, Q, R


A, B, Q, R = Linearize(x0, tau, "Double_Leg/Double_Leg_No_Coll.urdf")

print(controllability(A, B))

In [ ]:
# projection into null space of [[Jc 0], [Jc_dot, 0]] from Mason et al. 2016

# Function to get foot jacobain
def foot_jac(plant, context):

    frame_world = plant.world_frame()

    p_BoBi_B = np.zeros(3)

    for index, point in enumerate(feet_center):
        
        frame_foot = plant.GetFrameByName(point)

        Jc_sp = plant.CalcJacobianSpatialVelocity(
                context,
                JacobianWrtVariable(1),
                frame_foot,
                p_BoBi_B,
                frame_world,
                frame_world
                )

        if index == 0:
            Jc = Jc_sp
        else:
            Jc = np.vstack((Jc, Jc_sp))

    return Jc


def makeJc_dot(plant, context):

    frame_world = plant.world_frame()

    v = np.zeros(num_vel)
    p_BoBP_B = np.zeros(3)

    Jc_dot = np.zeros((12, num_vel))

    for col in range(num_vel):
        v[col] = 1
        plant.SetVelocities(context, v)
        
        for index, point in enumerate(feet_center):
            
            frame_B = plant.GetFrameByName(point)
            
            
            Sp_acc = plant.CalcBiasSpatialAcceleration(context,
                            JacobianWrtVariable(1),
                            frame_B,
                            p_BoBP_B,
                            frame_world,
                            frame_world)

            Jc_dot[0 + 6*index:3 + 6*index, col] = Sp_acc.rotational()
            Jc_dot[3 + 6*index:6 + 6*index, col] = Sp_acc.translational()

        v[col] = 0

    return Jc_dot

builder_temp = DiagramBuilder()
plant_temp, scene_graph_temp = AddMultibodyPlantSceneGraph(builder_temp, time_step=0.0)
body_temp = Parser(plant_temp).AddModelFromFile("Double_Leg/Double_Leg_Collisions.urdf")
plant_temp.Finalize()

# Update plant context
context_temp = plant_temp.CreateDefaultContext()
plant_temp.SetPositionsAndVelocities(context_temp, x0)
plant_temp.get_actuation_input_port().FixValue(context_temp, tau)

# Get Jc and Jc_dot
Jc = foot_jac(plant_temp, context_temp)
Jc_dot = makeJc_dot(plant_temp, context_temp)

# Make N matrix
N = np.tile(np.zeros(np.shape(Jc)), (2, 2))
N[:len(Jc[:, 0]), :len(Jc[0, :])] = N[len(Jc[:, 0]):, len(Jc[0, :]):] = Jc
N[len(Jc[:, 0]):, :len(Jc[0, :])] = Jc_dot

# Get the nullspace of the N matrix using Scipy's null_space function
null_N = null_space(N)

# Projected states
xm = null_N.T @ x0[1:]
Am = null_N.T @ A @ null_N
Bm = null_N.T @ B
Rm = R
Qm = null_N.T @ Q @ null_N

time_step = 1/1000.

# Discretize system (not compatible with Jc_dot most likely)
Am_disc = expm(time_step*(Am))
Bm_disc = np.linalg.inv(Am) @ (Am_disc - np.eye(len(Am))) @ Bm

Km, Sm = LinearQuadraticRegulator(Am, Bm, Qm, Rm)
Km_disc, Sm_disc = DiscreteTimeLinearQuadraticRegulator(Am_disc, Bm_disc, Qm, Rm)

lqr_gain = Km @ null_N.T     

print(Km - Km_disc)

print(controllability(Am, Bm))
print(stabilizability(Am, Bm, Km, "discrete"))


In [ ]:
# Wire contact detection logic
contact = builder.AddSystem(ReadContactResults(plant, foot_vertices))
lqr = builder.AddSystem(contact_LQR(num_pos+num_vel, num_act, lqr_gain, x0, tau))
ZOH = builder.AddSystem(ZeroOrderHold(time_step, 1))

# Connect all systems
builder.Connect(plant.get_contact_results_output_port(), contact.get_input_port())
# builder.Connect(contact.get_output_port(1), lqr.get_input_port(1))
builder.Connect(plant.get_state_output_port(), lqr.get_input_port(0))
builder.Connect(lqr.get_output_port(), plant.get_actuation_input_port())

builder.Connect(contact.get_output_port(1), ZOH.get_input_port())
builder.Connect(ZOH.get_output_port(), lqr.get_input_port(1))

# generic_input = builder.AddSystem(ConstantVectorSource(0.05*np.ones(num_act)))
# builder.Connect(generic_input.get_output_port(), plant.get_actuation_input_port())

logger_is_contact = LogVectorOutput(contact.get_output_port(1), builder, publish_period=1/1000)
logger_torques = LogVectorOutput(lqr.get_output_port(), builder, publish_period=1/1000)
diagram = builder.Build()

display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=2))[0].create_svg()))

In [ ]:
simulator = Simulator(diagram)
sim_context = simulator.get_mutable_context()

x0[6] += 0.3

sim_context.SetTime(0.)
# sim_context.SetDiscreteState(0, x0)
sim_context.SetContinuousState(x0)
vis.StartRecording()
simulator.AdvanceTo(2)
vis.StopRecording()
vis.PublishRecording()

meshcat.DeleteAddedControls()

contact_data = logger_is_contact.FindLog(sim_context).data().flatten()
torque_data = logger_torques.FindLog(sim_context).data()
t = np.arange(0, len(contact_data))

fig, ax = plt.subplots()
ax.plot(t, contact_data)

fig_shaft, ax_shaft = plt.subplots()
ax_shaft.plot(t, torque_data[0, :], label="shaft roll")
ax_shaft.plot(t, torque_data[1, :], label="shaft_pitch")
ax_shaft.legend()

fig_hips, ax_hips = plt.subplots()
ax_hips.plot(t, torque_data[2, :], label="hip roll R")
ax_hips.plot(t, torque_data[3, :], label="hip pitch R")
ax_hips.plot(t, torque_data[7, :], label="hip roll L")
ax_hips.plot(t, torque_data[8, :], label="hip pitch L")
ax_hips.legend()

fig_knees, ax_knees = plt.subplots()
ax_knees.plot(t, torque_data[4, :], label="knee R")
ax_knees.plot(t, torque_data[9, :], label="knee L")
ax_knees.legend()

fig_feet, ax_feet = plt.subplots()
ax_feet.plot(t, torque_data[5, :], label="foot pitch R")
ax_feet.plot(t, torque_data[6, :], label="foot roll R")
ax_feet.plot(t, torque_data[10, :], label="foot pitch L")
ax_feet.plot(t, torque_data[11, :], label="foot roll L")
ax_feet.legend()

plt.plot()